In [1]:
import os
import re
import sys
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import skimage
from skimage import io
from sklearn import preprocessing
from tqdm.notebook import tqdm, trange
import anndata as ad
import cv2
import scanorama
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
# Import spatial omics library
import athena as ath
from spatialOmics import SpatialOmics

# import default graph builder parameters
from athena.graph_builder.constants import GRAPH_BUILDER_DEFAULT_PARAMS

In [3]:
d_dir = (Path().cwd().parents[0].parents[0]).absolute()
data_dir = d_dir / "09_datasets"

p_dir = (Path().cwd().parents[0]).absolute()

In [4]:
%load_ext autoreload
%autoreload 2

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)


In [5]:
import graph
import torch
import torch_geometric.utils
import networkx as nx
import lightning.pytorch as pl
import torch.utils.data as data

spatial_omics_folder = (Path().cwd().parents[0]).absolute() / 'data' / 'spatial_omics_graph'
process_path = (Path().cwd().parents[0]).absolute() / 'data' / 'torch_graph_data'
morph_path = (Path().cwd().parents[0]).absolute() / 'data' / 'morph' / 'outlinePCA.pkl'

# Create data loader

In [6]:
from torch_geometric.loader import DataLoader
seed = torch.Generator().manual_seed(42)

name = 'NIH'

# Crate dataset
dataset = graph.GraphDatasetMorph(process_path / name, morph_path, process_path / name / 'info.csv', 2, y_name='covid')

train_set, val_set, test_set = graph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [7]:
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.nc}')

Dataset: GraphDatasetMorph(442):
Number of graphs: 442
Number of features: 5
Number of classes: 2


In [8]:
print(f'Train set: {len(train_set)}, val set: {len(test_set)}, val set: {len(val_set)}')

Train set: 213, val set: 176, val set: 53


In [9]:
for step, data in enumerate(test_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    data.label
    break

Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 887767], num_nodes=127015, x=[127015, 5], pos=[127015, 2], node_types=[127015], label=[32], covid=[32], train_mask=[127015], test_mask=[127015], y=[32], name=[32], features=[32, 34], features_names=[32], batch=[127015], ptr=[33])



# Train network K Fold

In [10]:
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [11]:
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit
from torch.utils.data import SubsetRandomSampler

In [12]:
# # Print K fold model number of samples and number of positive cases

# k_folds = 5
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset)):
#     train_subset = dataset.index_select(train_ids.tolist())
#     val_subset = dataset.index_select(valid_ids.tolist())
    
#     train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
#     val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
    
#     train_positives = 0
#     test_positives = 0
#     for step, data in enumerate(train_loader):
#         train_positives += np.sum(data.y.numpy())
#     for step, data in enumerate(val_loader):
#         test_positives += np.sum(data.y.numpy())
        
#     print(len(train_subset), train_positives, len(val_subset), test_positives)

In [13]:
condition = '082423_Covid'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f"Graph_GNNs_moprh_{condition}" 
project_name = f'SF_{condition}'

In [14]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 2
HIDDEN_CHANNELS = 16
# pools = ['mean', 'sum', 'max', 'attention', 'attention2']
# models = ['GCN', 'GraphConv', 'GAT', 'GINConv', 'SAGEConv']
pools = ['mean', 'max', 'attention', 'attention2']
models = ['MLP']
epochs = 100

INFO: Global seed set to 42
INFO:lightning.fabric.utilities.seed:Global seed set to 42


In [15]:
ys = [data.y.item() for data in dataset]

In [16]:
# Print K fold model number of samples and number of positive cases

k_folds = 5
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
kfold = StratifiedShuffleSplit(n_splits=k_folds, test_size=0.3, random_state=0)
fusion_mode = 'concat'

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset, ys)):
    train_subset = dataset.index_select(train_ids.tolist())
    val_subset = dataset.index_select(valid_ids.tolist())
    
    for pool in pools:
        for model in models:
            # Path to the folder where the pretrained models are saved
            CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}_moprh_{fusion_mode}' / pool
            CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

            # Skip already trained kfold and pool
            checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
            if checkpoint.exists():
                print(checkpoint)
                continue

            # Run training
            run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}_moprh', 
                            group=f'{model}_{pool}_moprh_{fusion_mode}')
            graph.train_graph_classifier_kfold(model, 
                                                 train_subset, 
                                                 val_subset, 
                                                 dataset, 
                                                 CHECKPOINT_PATH, 
                                                 AVAIL_GPUS, 
                                                 hidden_channels=HIDDEN_CHANNELS, 
                                                 num_layers=NUM_LAYERS, 
                                                 epochs=epochs,
                                                 morph=True,
                                                 fusion = fusion_mode,
                                                 batch_size=128,
                                                 graph_pooling=pool)
            run.finish()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


INFO: Global seed set to 42
INFO:lightning.fabric.utilities.seed:Global seed set to 42
C:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Global seed set to 42
INFO:lightning.fabric.utilities.seed:Global seed set to 42


Training with Morphological features


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: 
   | Name           | Type             | Params
-----------------------------------------------------
0  | feat_embedding | Sequential       | 832   
1  | model          | MLPModel         | 368   
2  | head           | Sequential       | 562   
3  | fnn_layer      | Linear           | 272   
4  | selu           | SELU             | 0     
5  | loss_module    | CrossEntropyLoss | 0     
6  | train_acc      | BinaryAccuracy   | 0     
7  | train_auroc    | BinaryAUROC      | 0     
8  | train_f1       | BinaryF1Score    | 0     
9  | valid_acc      | BinaryAccuracy   | 0     
10 | valid_auroc    | BinaryAUROC      | 0     
11 | valid_f1       | BinaryF1Score    | 0     
-----------------------------------------------------
2.0 K     Trainable params
0         Non-trainable params
2.0 K     Total params
0.008     Total estimated model params size (

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▂▂▃▃▃▃▄▄▄▄▄▅▅▆▆▆▇▇▇▇█▇▇▇▇█▇▇▇▇█▇▇▇▇█▇▇
train_auc,▁▁▂▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇█▇█▇████████▇█▇████
train_f1,▁▂▂▃▃▃▃▂▄▄▃▄▄▄▄▆▅▅▆▇▇▇▇▇▆▇▇█▇▇▇▇█▇▇▆▇█▇▇
train_loss_epoch,████▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▂▂▂▃▂▂▂▂▃▂▂▂▂▂▂▂▁▂▂
train_loss_step,▇▇█▇▆▄▆▆█▄▅▅▅▃▂▂▃▄▂▃▁▂▃▃▄▃▃▂▂▂
trainer/global_step,▁▁▁▂▁▂▂▂▃▃▃▃▃▃▄▄▄▄▅▃▄▄▅▅▆▄▄▄▆▇▇▅▅▅███▆▆▆
val_acc,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▅▅▆▅▇▆▇▆▆█▇▇▇▇▇▇▆▇▆█▆▇▇
val_auc,▁▂▃▃▃▃▂▃▃▃▃▃▃▄▄▄▅▅▆▇▇███▇████████▇██████
val_f1,▃▃▃▃▂▂▁▁▃▃▁▃▂▂▃▅▄▅▅▆▅▆▆▇▅▆█▆▆▇▇▇▇▄▇▄█▅▇▆


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▂▂▂▃▃▃▃▄▅▄▅▄▄▄▅▆▅▅▅▆▅▆▇▇▇▇▇█████▇█▇▇▇██
train_auc,▁▂▂▂▃▃▄▄▅▄▄▅▅▅▅▅▅▆▆▇▆▆▇▇▇█▇████████▇▇███
train_f1,▁▂▃▂▃▃▂▄▃▅▄▅▅▄▄▄▅▅▄▃▄▄▆▆▆▇▇▇▇███▇▇▇▅▆▇█▇
train_loss_epoch,████▇▇▇▇▆▅▆▅▆▅▅▅▄▄▄▄▄▅▃▃▃▂▃▂▂▂▁▁▂▂▂▃▂▂▁▁
train_loss_step,▇██▇▇▆▅▇▆▅▄▃▄▅▃▅▃▄▃▂▂▃▃▂▂▁▄▃▂▁
trainer/global_step,▁▁▁▂▁▂▂▂▃▃▃▃▃▃▄▄▄▄▅▃▄▄▅▅▆▄▄▄▆▇▇▅▅▅███▆▆▆
val_acc,▁▁▁▃▅▄▅▂▄▄▄▄▃▃▄▅▆▇▇▇▇▇▇▆▇▇▇▆▇▇▇▇▆█▆▆▆██▅
val_auc,▁▂▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▆▇▆▇▇▇▇▇▇██████████
val_f1,▂▂▂▃▄▄▅▃▄▃▄▄▃▃▄▅▆▇▇▆▇▇▇▄▆▆▇▅▇▆▇▆▃█▄▄▅▇█▁


INFO: Global seed set to 42
INFO:lightning.fabric.utilities.seed:Global seed set to 42
C:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Global seed set to 42
INFO:lightning.fabric.utilities.seed:Global seed set to 42
INFO: LO

Training with Morphological features


C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\lightning\pytorch\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
C:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kw

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▁▂▂▂▃▃▄▄▄▄▅▄▅▅▅▅▆▆▇▇▇▇█▇█▇▇████▇██████
train_auc,▁▁▂▂▄▃▄▄▄▄▄▅▅▅▅▅▆▅▆▆▆▇▇▇▇▇██▇███████████
train_f1,▁▁▂▂▃▂▂▃▃▄▃▄▄▃▅▄▅▅▆▆▆▆▇▇█▆▇▇▅████▆█▇▇██▇
train_loss_epoch,████▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▂▃▂▂▂▂▃▂▁▁▁▂▂▂▂▂▂▂
train_loss_step,███▇▇▇▅▆▅▇▅▅▅▅▄▃▃▄▄▃▄▃▃▁▄▂▃▁▂▃
trainer/global_step,▁▁▁▂▁▂▂▂▃▃▃▃▃▃▄▄▄▄▅▃▄▄▅▅▆▄▄▄▆▇▇▅▅▅███▆▆▆
val_acc,▁▁▁▁▁▄▄▄▄▄▄▄▄▅▅▆▅▆▆▆▆▇▇▇▇█▇▆▇▇▇▇▇▆██▇▇█▇
val_auc,▁▃▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇█████████████████
val_f1,▁▁▁▁▁▃▂▃▃▃▄▄▄▅▄▅▅▅▅▆▆▇▆▇▆█▆▆▆▇▆▇▇▅██▇▇█▆
